In [2]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Set paths
train_dataset_path = r'D:\Rice-Disease-Simulation\data\train'
val_dataset_path = r'D:\Rice-Disease-Simulation\data\validation'
img_size = (224, 224)

# Load and preprocess training images
def load_images_and_labels(dataset_path):
    images = []
    labels = []
    for class_name in os.listdir(dataset_path):
        class_path = os.path.join(dataset_path, class_name)
        if not os.path.isdir(class_path): continue
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            img = cv2.imread(img_path)
            if img is None:
                print(f"[WARNING] Couldn't read {img_path}")
                continue
            img = cv2.resize(img, img_size)
            img = preprocess_input(img)
            images.append(img)
            labels.append(class_name)
    return np.array(images, dtype=np.float32), np.array(labels)

# Load training data
train_images, train_labels = load_images_and_labels(train_dataset_path)
print(f"[INFO] Loaded training images: {train_images.shape}, labels: {train_labels.shape}")

# Load MobileNetV2 for feature extraction
base_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')

# Extract features
train_features = base_model.predict(train_images, batch_size=32, verbose=1)
print(f"[INFO] Extracted training features: {train_features.shape}")

# Encode labels
le = LabelEncoder()
train_labels_encoded = le.fit_transform(train_labels)

# Scale features
scaler = StandardScaler()
train_features_scaled = scaler.fit_transform(train_features)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    train_features_scaled, train_labels_encoded,
    test_size=0.2, stratify=train_labels_encoded, random_state=42
)

# Train SVM
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale')
svm_model.fit(X_train, y_train)

# Evaluate on test split
y_pred_test = svm_model.predict(X_test)
print("\n[INFO] Test Set Classification Report:")
print(classification_report(y_test, y_pred_test, target_names=le.classes_))

[INFO] Loaded training images: (2100, 224, 224, 3), labels: (2100,)


C:\Users\uzmap\AppData\Local\Temp\ipykernel_33940\3456836568.py:40: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')


66/66 ━━━━━━━━━━━━━━━━━━━━ 16s 228ms/step
[INFO] Extracted training features: (2100, 1280)

[INFO] Test Set Classification Report:
                       precision    recall  f1-score   support

bacterial_leaf_blight       0.97      1.00      0.99        70
           brown_spot       0.83      0.96      0.89        70
              healthy       0.97      0.93      0.95        70
           leaf_blast       0.90      0.80      0.85        70
           leaf_scald       0.97      0.99      0.98        70
    narrow_brown_spot       1.00      0.96      0.98        70

             accuracy                           0.94       420
            macro avg       0.94      0.94      0.94       420
         weighted avg       0.94      0.94      0.94       420



untouched validation set

In [3]:
# Extract validation features
def extract_features_from_folder(folder_path):
    images = []
    labels = []
    for class_name in os.listdir(folder_path):
        class_path = os.path.join(folder_path, class_name)
        if not os.path.isdir(class_path): continue
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            img = cv2.imread(img_path)
            if img is None:
                print(f"[WARNING] Couldn't read {img_path}")
                continue
            img = cv2.resize(img, img_size)
            img = preprocess_input(img)
            images.append(img)
            labels.append(class_name)
    images = np.array(images, dtype=np.float32)
    print(f"[INFO] Extracted {len(images)} validation images with shape: {images.shape}")
    features = base_model.predict(images, batch_size=32, verbose=1)
    print(f"[INFO] Validation feature shape: {features.shape}")
    return features, labels

# Run validation
X_val, y_val = extract_features_from_folder(val_dataset_path)
X_val_scaled = scaler.transform(X_val)
y_val_encoded = le.transform(y_val)

# Predict on validation
y_pred_val = svm_model.predict(X_val_scaled)
print("\n[INFO] Validation Set Classification Report:")
print(classification_report(y_val_encoded, y_pred_val, target_names=le.classes_))

[INFO] Extracted 528 validation images with shape: (528, 224, 224, 3)
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 189ms/step
[INFO] Validation feature shape: (528, 1280)

[INFO] Validation Set Classification Report:
                       precision    recall  f1-score   support

bacterial_leaf_blight       0.99      1.00      0.99        88
           brown_spot       0.87      0.90      0.88        88
              healthy       0.90      0.93      0.92        88
           leaf_blast       0.84      0.78      0.81        88
           leaf_scald       0.99      0.99      0.99        88
    narrow_brown_spot       0.98      0.97      0.97        88

             accuracy                           0.93       528
            macro avg       0.93      0.93      0.93       528
         weighted avg       0.93      0.93      0.93       528

